# Loop 11 Analysis: Strategic Decision

## Key Questions:
1. Should we submit exp_011 (CV=0.82032)?
2. What's the expected LB based on CV-LB relationship?
3. What should be the next experiment?

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

# Submission history
submissions = [
    {'exp': 'exp_000', 'cv': 0.80674, 'lb': 0.79705},
    {'exp': 'exp_003', 'cv': 0.81951, 'lb': 0.80453},
    {'exp': 'exp_004', 'cv': 0.81928, 'lb': 0.80406},
    {'exp': 'exp_006', 'cv': 0.81709, 'lb': 0.80102},
]

df = pd.DataFrame(submissions)
df['gap'] = df['cv'] - df['lb']
print('Submission History:')
print(df.to_string(index=False))
print(f'\nMean CV-LB gap: {df["gap"].mean():.5f}')
print(f'Gap range: {df["gap"].min():.5f} - {df["gap"].max():.5f}')

In [ ]:
# Linear regression to predict LB from CV
from scipy.stats import linregress

slope, intercept, r_value, p_value, std_err = linregress(df['cv'], df['lb'])
print(f'CV-LB Relationship: LB = {slope:.4f} * CV + {intercept:.4f}')
print(f'R-squared: {r_value**2:.4f}')

# Predict LB for exp_011
exp_011_cv = 0.82032
predicted_lb = slope * exp_011_cv + intercept
print(f'\nexp_011 CV: {exp_011_cv:.5f}')
print(f'Predicted LB: {predicted_lb:.5f}')
print(f'Best LB so far: 0.80453')
print(f'Expected improvement: {predicted_lb - 0.80453:.5f}')

In [ ]:
# Alternative: Use mean gap to predict
mean_gap = df['gap'].mean()
predicted_lb_gap = exp_011_cv - mean_gap
print(f'Using mean gap ({mean_gap:.5f}):')
print(f'Predicted LB: {predicted_lb_gap:.5f}')

# What CV do we need to beat 0.80453?
target_lb = 0.80453
required_cv_regression = (target_lb - intercept) / slope
required_cv_gap = target_lb + mean_gap
print(f'\nTo beat LB 0.80453:')
print(f'  Using regression: CV > {required_cv_regression:.5f}')
print(f'  Using mean gap: CV > {required_cv_gap:.5f}')
print(f'\nexp_011 CV (0.82032) vs threshold ({required_cv_regression:.5f}): {"ABOVE" if exp_011_cv > required_cv_regression else "BELOW"}')

In [ ]:
# Key insight: The CV-LB gap is INCREASING with higher CV
print('CV-LB Gap Analysis:')
for _, row in df.iterrows():
    print(f"  {row['exp']}: CV={row['cv']:.5f}, Gap={row['gap']:.5f}")

# Correlation between CV and gap
corr = np.corrcoef(df['cv'], df['gap'])[0, 1]
print(f'\nCorrelation between CV and Gap: {corr:.4f}')
if corr > 0.5:
    print('WARNING: Higher CV tends to have higher gap (more overfitting)')
    print('This suggests exp_011 may have even larger gap than predicted')

In [ ]:
# Decision matrix
print('\n=== DECISION MATRIX ===')
print('\nOption 1: Submit exp_011 now')
print(f'  - Best CV ever (0.82032)')
print(f'  - Predicted LB: {predicted_lb:.5f} (regression) or {predicted_lb_gap:.5f} (mean gap)')
print(f'  - Risk: Gap may be larger due to CV-gap correlation')
print(f'  - Benefit: Get LB feedback on regularization approach')

print('\nOption 2: Try GroupKFold first')
print('  - May reduce variance and give more realistic CV')
print('  - Could reveal if exp_011 CV is inflated')
print('  - Delays LB feedback')

print('\nOption 3: Try KNN imputation first')
print('  - Different data preprocessing (not model changes)')
print('  - Used by top solutions (0.8066 LB)')
print('  - Could provide breakthrough')

print('\n=== RECOMMENDATION ===')
print('Submit exp_011 NOW because:')
print('1. Best CV ever achieved')
print('2. We have 6 submissions remaining - can afford to test')
print('3. Need LB feedback to calibrate regularization approach')
print('4. Even if LB is slightly below best, we learn about CV-LB relationship')